In [1]:
sc.master

'local[*]'

In [6]:
stu_rdd = sc.textFile("inputData/student.txt").map(lambda x:x.split("\t"))

In [7]:
bigdata_rdd = sc.textFile("inputData/result_bigdata.txt").map(lambda x:x.split("\t"))

In [13]:
math_rdd = sc.textFile("inputData/result_math.txt").map(lambda x:x.split("\t"))

In [12]:
bigdata_rdd.sortBy(lambda x:int(x[2]),ascending=False).take(5)

[['1003', '大数据基础', '100'],
 ['1007', '大数据基础', '100'],
 ['1004', '大数据基础', '99'],
 ['1002', '大数据基础', '94'],
 ['1006', '大数据基础', '94']]

In [14]:
math_rdd.sortBy(lambda x:int(x[2]),ascending=False).take(5)

[['1003', '应用数学', '100'],
 ['1004', '应用数学', '100'],
 ['1001', '应用数学', '96'],
 ['1002', '应用数学', '94'],
 ['1005', '应用数学', '94']]

In [19]:
stu_info = stu_rdd.map(lambda x:(x[0],x[1])).join(bigdata_rdd.map(lambda x:(x[0],[x[1],x[2]])).join(math_rdd.map(lambda x:(x[0],[x[1],x[2]]))))

In [20]:
stu_info.take(2)

[('1001', ('李正明', (['大数据基础', '90'], ['应用数学', '96']))),
 ('1004', ('张永丽', (['大数据基础', '99'], ['应用数学', '100'])))]

In [24]:
stu_info1 = stu_info.map(lambda x:[x[0],x[1][0],x[1][1][0][0],x[1][1][0][1],x[1][1][1][0],x[1][1][1][1]])

In [25]:
def findid(x):
    if(x[3]=="100" or x[5]=="100"):
        return x[0]

In [27]:
stu_info1.map(lambda x:findid(x)).filter(lambda x:x != None).collect()

['1004', '1003', '1007']

In [28]:
def sumgrade(x):
    sum = int(x[3]) + int(x[5])
    return sum

In [29]:
stu_info1.map(lambda x:(x[0],sumgrade(x))).collect()

[('1001', 186),
 ('1004', 199),
 ('1008', 187),
 ('1012', 175),
 ('1002', 188),
 ('1003', 200),
 ('1005', 184),
 ('1006', 174),
 ('1009', 173),
 ('1007', 190),
 ('1010', 164),
 ('1011', 170)]

In [30]:
def avefgrade(x):
    sum = int(x[3]) + int(x[5])
    avggrade = sum / 2.0
    return avggrade

In [32]:
stu_info1.map(lambda x:(x[0],avefgrade(x))).collect()

[('1001', 93.0),
 ('1004', 99.5),
 ('1008', 93.5),
 ('1012', 87.5),
 ('1002', 94.0),
 ('1003', 100.0),
 ('1005', 92.0),
 ('1006', 87.0),
 ('1009', 86.5),
 ('1007', 95.0),
 ('1010', 82.0),
 ('1011', 85.0)]

In [33]:
stu_info1.map(lambda x:[x[0],x[1],x[3],x[5],sumgrade(x),avefgrade(x)]).collect()

[['1001', '李正明', '90', '96', 186, 93.0],
 ['1004', '张永丽', '99', '100', 199, 99.5],
 ['1008', '沈彬', '93', '94', 187, 93.5],
 ['1012', '钱多多', '84', '91', 175, 87.5],
 ['1002', '王一磊', '94', '94', 188, 94.0],
 ['1003', '陈志华', '100', '100', 200, 100.0],
 ['1005', '赵信', '90', '94', 184, 92.0],
 ['1006', '古明远', '94', '80', 174, 87.0],
 ['1009', '李子琪', '89', '84', 173, 86.5],
 ['1007', '刘浩明', '100', '90', 190, 95.0],
 ['1010', '王嘉栋', '78', '86', 164, 82.0],
 ['1011', '柳梦文', '91', '79', 170, 85.0]]

In [18]:
room_info = spark.read.format("csv").option("dilimiter",",").option("header","true").load("inputData/58fang.txt")

In [34]:
df = room_info.grboupBy("区名").agg({"房间数":"sum"})

In [35]:
df.select(df["区名"],df["sum(房间数)"].alias("number").astype("int")).show()

+----+------+
|  区名|number|
+----+------+
|  长沙|     5|
|  芙蓉|  1160|
|  开福|  1726|
|  望城|   297|
|  天心|  1442|
|  岳麓|  2511|
|  星沙|  1455|
|长沙周边|   556|
|  雨花|  3554|
+----+------+



In [36]:
room_info.show()

+---+-----+---+----+------+--------+
|房间数|   面积|楼层数|  区名|是否邻近地铁|均价（元/m2）|
+---+-----+---+----+------+--------+
|  9|152.0| 28|  雨花|     0|   13684|
|  8|157.0| 11|  雨花|     0|   11210|
|  9| 91.0|  7|  雨花|     0|    8571|
|  5| 90.0| 30|  雨花|     0|   10200|
|  7|108.0| 32|  星沙|     0|    9074|
|  3| 40.0| 20|  星沙|     0|    6000|
|  5| 94.0| 33|  星沙|     0|    7000|
|  6| 91.0| 22|长沙周边|     0|    6791|
|  7|126.0| 15|  岳麓|     0|   10793|
|  7|122.7| 32|  岳麓|     1|   13528|
|  8|138.0| 32|  岳麓|     0|   10144|
|  6| 88.0|  9|  天心|     0|    6590|
|  8|142.0| 34|  岳麓|     0|   13028|
|  6| 90.5| 33|  岳麓|     1|    6464|
|  7|141.0| 33|  开福|     1|   12127|
| 11|302.0| 56|  天心|     1|   18807|
|  8|110.0| 33|  星沙|     0|    9818|
|  3| 30.0| 31|  雨花|     0|    7000|
|  7|114.0| 15|  天心|     1|   10877|
|  6| 90.0| 33|  星沙|     0|    8333|
+---+-----+---+----+------+--------+
only showing top 20 rows



In [39]:
df1 = room_info.select("区名","面积",(room_info["面积"]*room_info["均价（元/m2）"]).alias("price"))

In [43]:
df1 = df1.groupBy("区名").agg({"面积":"sum","price":"sum"})

In [44]:
df1.show()

+----+--------------------+------------------+
|  区名|          sum(price)|           sum(面积)|
+----+--------------------+------------------+
|  长沙|            859918.0|              89.0|
|  芙蓉|2.0741558197000003E8|           20401.7|
|  开福|      3.8074137603E8|32231.239999999998|
|  望城|       6.399827111E7|           6601.27|
|  天心|      2.8392913231E8|          26677.34|
|  岳麓|      5.4795575103E8|48214.219999999994|
|  星沙|2.4895184532999995E8|28106.530000000006|
|长沙周边| 7.411110357000001E7|11219.269999999999|
|  雨花|      6.8685247977E8|          64084.77|
+----+--------------------+------------------+



In [56]:
df2 = df1.select(df["区名"],df1["sum(面积)"].alias("totalArea").cast("float"),df1["sum(price)"].alias("totalPrice").cast("float"))

In [57]:
df2.select("区名",(df2["totalPrice"] / df2["totalArea"]).alias("AvgPrice")).show()

+----+------------------+
|  区名|          AvgPrice|
+----+------------------+
|  长沙|            9662.0|
|  芙蓉|10166.583762267044|
|  开福|11812.805626819621|
|  望城| 9694.842327407849|
|  天心|10643.082168723779|
|  岳麓|11365.024472163619|
|  星沙| 8857.437977149299|
|长沙周边| 6605.697794635555|
|  雨花|10717.873919560036|
+----+------------------+



In [3]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.master("local").appName("fy").getOrCreate()
raw_df = spark.read.format("csv").option("dilimiter",",").option("header","true").load("d://studyPython/inputData/women.csv")
df = raw_df.select([col(column).cast("float") for column in raw_df.columns[1:3]])
assemblerInput = ["height"]
assembler = VectorAssembler(inputCols=assemblerInput,outputCol="features")
model = LinearRegression(featuresCol="features",labelCol="weight")
pipeline = Pipeline(stages=[assembler,model])
pipelineModel = pipeline.fit(df)